DROPS2 Interactive Notebook 🚀
-----------------

## Tips:
- press **shift+return** to run a cell and advance to the next, or **ctrl+return** to run a cell
- access the docs for a function using **?** e.g. :
        coverages.get_data?
- use _**tab**_ to get the auto-completion while editing a cell 

enjoy 😉

------------------------------------------------------------------
** Useful docs 📖 ** 

[xarray doc](http://xarray.pydata.org/en/stable/)

[pandas doc](http://pandas.pydata.org/pandas-docs/stable/10min.html)

[geopandas doc](http://geopandas.org/)

[numpy doc](https://docs.scipy.org/doc/numpy-1.13.0/reference/)

[jupyter doc](http://jupyter.org/documentation.html)

[pyplot doc](https://matplotlib.org/api/pyplot_api.html)

------------------------------------------------------------------

In [1]:
# import the drops2 module
from drops2 import sensors, coverages

In [2]:
# import all the plot functions and make them plot inline
%pylab inline
import pandas as pd
import geopandas as gpd
import xarray as xr
import numpy as np
from datetime import datetime, timedelta
from IPython.lib.display import FileLink

Populating the interactive namespace from numpy and matplotlib



Create a map for the cumulated precipitation
------------------------------------------------

In [3]:
date_from = '201712110000'
date_to = '201712120000'
sensor_class = 'PLUVIOMETRO'

In [4]:
#require the sensor list within the default geowindow
sensors_list = sensors.get_sensor_list(sensor_class, geo_win=(6.0, 36.0, 18.6, 47.5))
df_sensors = sensors_list.to_geopandas()
df_sensors.head()

,name,mu,geometry
id,,,
1073800617_1,Rio Gere,mm,POINT (12.186649 46.56783)
1073800618_1,Rio Gere,mm,POINT (12.186649 46.56783)
58795_1,Mezzolombardo,mm,POINT (11.103056 46.19389)
268442973_2,Acqui Terme,mm,POINT (8.463333 44.678333)
-2147444435_2,Lecce,mm,POINT (18.178 40.354332)


In [ ]:
# get the data as a pandas dataframe
df_pluvio = sensors.get_sensor_data(sensor_class, sensors_list, date_from, date_to, as_pandas=True)
# filter out the invalid values
df_pluvio[df_pluvio<0] = np.NaN


In [ ]:
# combine the sensor dataframe with the cumulate from the extracted values
df_sensors['value'] = df_pluvio.sum()

In [ ]:
# plot values as choropleth map
df_sensors.sort_values('value').plot(column='value', cmap='jet', markersize=5, legend=True)

In [ ]:
# get the 5 sensors with highest cumulate
df_sensors.sort_values('value', ascending=False).head()


Download of a timeserie for a selected sensor
------------------------------------------------

In [ ]:
date_from = '201712100000'
date_to = '201712130000'
sensor_class = 'PLUVIOMETRO'

# Selected sensor id
sensor_id = '-2147433690_1'

In [ ]:
#require the sensor list within the default geowindow
sensors_list = sensors.get_sensor_list(sensor_class, geo_win=(6.0, 36.0, 18.6, 47.5))

# take the selected sensor by id
sensor = sensors_list[sensor_id]
print(sensor)

In [ ]:
# get the data as dataframe
df_values = sensors.get_sensor_data(sensor_class, [sensor], date_from, date_to, as_pandas=True)

In [ ]:
# create a file and access it using a download link
filename = 'data.csv'
df_values.to_csv(filename)
FileLink(path=filename)

In [ ]:
# resample the data by cumulating on 3H window
df_resampled = df_values.resample('3H').sum()

# bar plot of the cumulated precipitation
df_resampled.plot.bar(figsize=(20,10), rot=45)
grid()

Get a sensor map
---------------

In [ ]:
# generate a date range list
date_range = [datetime(2017, 12, 10) + timedelta(days=d) for d in range(4)]

# get the maps from the service using a 24h cum time
cum_hours = 24
sensor_class = 'PLUVIOMETRO'
data = sensors.get_sensor_map(sensor_class, date_range, 
                              cum_hours=cum_hours, geo_win=(6.0, 36.0, 18.6, 47.5),
                              img_dim=(630, 575), radius=0.5)


In [ ]:
# generate the variable name
variable_name = sensor_class + '_' + str(cum_hours) + 'h'

# fix missing values
P = data[variable_name]
P.values[P.values<0] = NaN


In [ ]:
# plot the data
for d in date_range:
    figure()
    imshow(flipud(P.loc[d,:,:]))
    colorbar()
    title(d)

Download data from COSMO LAMI
---------------

In [ ]:
# print all the supported models
models = coverages.get_supported_data()

for m in models[0:5]:
    print(m)

In [ ]:
# let's get the available reference dates for the selected model
data_id = 'COSMO_LAMI_I7'
model_dates = coverages.get_dates(data_id, date_from, date_to)
print('the first 5 data ids')
for d in model_dates[0:5]:
    print(d)
print('...')    

In [ ]:
# select the first date and get the available variables
date_ref = model_dates[0]
variables = coverages.get_variables(data_id, date_ref)
print('the first 5 variables for ' + data_id)
for v in variables[0:5]: 
    print(v)
print('...')    

In [ ]:
# print the available levels for the selected variable
variable = 'LAMI_11'
levels = coverages.get_levels(data_id, date_ref, variable)
print('the first 5 levels for ' + data_id + ' ' + variable)
for lev in levels:
    print(lev)
print('...')        

In [ ]:
# print the timeline
level = '2'
timeline = coverages.get_timeline(data_id, date_ref, variable, level)

for t in timeline:
    print(t)

In [ ]:
# select all the timesteps for the model
date_selected = 'all'

# data is an xarray object 
data = coverages.get_data(data_id, date_ref, variable, level, date_selected=date_selected)

In [ ]:
# create a file and access it using a download link
filename = 'dataset.nc'
data.to_netcdf(filename)
FileLink(path=filename)

In [ ]:
imshow(flipud(data.LAMI_11.mean('time')))
colorbar()